# Predicting St. Vrain Hydrology using KNN Model

This notebooks allows users to utilize a pre-trained K-Nearest Neighbors Model for predicting St. Vrain Hydrology based on other Natural Flow nodes.  

Note: this notebook does not train a new model each time - but instead utilizes a pre-trained Reclamation model distributed with this notebook.

Please press Shift+Enter (at the sametime) to run each code cell in this notebook.

#### The cell below loads in the required Python libraries for running this notebook. If you get an import error please contact Zack to help you (zleady@usbr.gov).

In [1]:
# standard libraries
import os
import pickle
# data read-in / manipulation libraries
import pandas as pd
import numpy as np
# model building libraries
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
#from sklearn.neighbors import KNeighborsRegressor
# graphing libraries
import plotly.graph_objects as go

#### The cells below loads in the pre-trained saved model from the model.pkl file contained in the same folder as this notebook.

In [2]:
# modify this cell only if you have moved the K-NN model file from the default directory
model_filepath = r"" # <-- user input or default to "./KNNmodel_v1.pkl"
if not model_filepath:
    model_filepath = "./KNNmodel_v1.pkl"

In [3]:
# load saved scikit-learn K-NN Model 
with open(model_filepath, 'rb') as f:
    KNN_UC_model = pickle.load(f)

In [4]:
display(KNN_UC_model)

KNeighborsRegressor(n_neighbors=12, weights='distance')

#### The cells below read-in the user hydrology input - in this case it should be UpperColoradoRiver.Inflow Annual Natural Flow values

In [5]:
input_hydrology_filepath = "./test_nf_input.xlsx" # <---user must fill-in
# if you use an excel file please provide the sheetname
sheetname = "Sheet1" # <---user must fill-in if they use an Excel sheet file with more than 1 sheet

In [6]:
# read-in user data
# user data should be formatted as an *.csv or *.xlsx file - *.csv is preferred
if input_hydrology_filepath:
    ext = os.path.basename(input_hydrology_filepath).split(".")[1]
    if ext == "csv":
        input_df = pd.read_csv(input_hydrology_filepath, header=0, index_col=0)
    elif ext == "xlsx":
        input_df = pd.read_excel(input_hydrology_filepath, sheet_name=sheetname, header=0, index_col=0)
    else:
        print("Warning you must use a csv or xlsx file - macros are not supported")

In [7]:
display(input_df)

,NF_1,NF_2,NF_3
Year,,,
1906,149314.2240,149314.2240,149314.2240
1907,189603.2858,189603.2858,189603.2858
1908,91389.0603,91389.0603,91389.0603
1909,165939.3530,165939.3530,165939.3530
1910,76251.6143,76251.6143,76251.6143
...,...,...,...
2016,109972.0670,109972.0670,109972.0670
2017,139323.9440,139323.9440,139323.9440
2018,83574.9239,83574.9239,83574.9239


#### In order to use the K-NN model correctly the data must be Standard-scaled from scikit-learn (mean = 0)

In [8]:
# values must be scaled using StandardScaler
SS = StandardScaler()
SS_df = pd.DataFrame(SS.fit_transform(input_df),
                     columns=input_df.columns,
                     index=input_df.index)
display(SS_df)

,NF_1,NF_2,NF_3
Year,,,
1906,0.873931,0.873931,0.873931
1907,2.054987,2.054987,2.054987
1908,-0.824119,-0.824119,-0.824119
1909,1.361289,1.361289,1.361289
1910,-1.267867,-1.267867,-1.267867
...,...,...,...
2016,-0.279367,-0.279367,-0.279367
2017,0.581070,0.581070,0.581070
2018,-1.053187,-1.053187,-1.053187


#### The cells below run KNN model predictions from UpperColoradoRiver.Inflow Annual Natural Flow to St. Vrain Annual Natural Flow

In [15]:
# predict values
trace_dict = {}
for trace_col in SS_df.columns.tolist():
    trace_vals = SS_df[trace_col].values
    trace_valsr = trace_vals.reshape(-1, 1)
    print(f"Input original shape: {trace_vals.shape}, Input reshaped for model: {trace_valsr.shape}")
    trace_predict = KNN_UC_model.predict(trace_valsr)
    trace_predictr = trace_predict.flatten()
    print(f"Output from model shape: {trace_predict.shape}, Ouput dataframe shape: {trace_predictr.shape}")
    trace_dict[trace_col] = trace_predictr
    print("\n")

# create output dataframe of predicted St.Vrain values
output_stvrain_df = pd.DataFrame.from_dict(trace_dict)

Input original shape: (115,), Input reshaped for model: (115, 1)
Output from model shape: (115, 1), Ouput dataframe shape: (115,)


Input original shape: (115,), Input reshaped for model: (115, 1)
Output from model shape: (115, 1), Ouput dataframe shape: (115,)


Input original shape: (115,), Input reshaped for model: (115, 1)
Output from model shape: (115, 1), Ouput dataframe shape: (115,)




In [10]:
display(output_stvrain_df)

,NF_1,NF_2,NF_3
0,0.624332,0.624332,0.624332
1,1.396647,1.396647,1.396647
2,-0.695214,-0.695214,-0.695214
3,0.238154,0.238154,0.238154
4,-1.274984,-1.274984,-1.274984
...,...,...,...
110,0.007880,0.007880,0.007880
111,1.110410,1.110410,1.110410
112,-0.010443,-0.010443,-0.010443
113,-0.074925,-0.074925,-0.074925


#### Before writing out the predicted St. Vrain Annual Natural Flow the data must be Un-Scaled

In [11]:
SS_inverse = pd.DataFrame(SS.inverse_transform(output_stvrain_df),
                          columns=input_df.columns,
                          index=input_df.index)
display(SS_inverse)
display(input_df)

,NF_1,NF_2,NF_3
Year,,,
1906,140799.703118,140799.703118,140799.703118
1907,167145.500715,167145.500715,167145.500715
1908,95786.360849,95786.360849,95786.360849
1909,127626.128203,127626.128203,127626.128203
1910,76008.833726,76008.833726,76008.833726
...,...,...,...
2016,119770.836640,119770.836640,119770.836640
2017,157381.176654,157381.176654,157381.176654
2018,119145.810388,119145.810388,119145.810388


,NF_1,NF_2,NF_3
Year,,,
1906,149314.2240,149314.2240,149314.2240
1907,189603.2858,189603.2858,189603.2858
1908,91389.0603,91389.0603,91389.0603
1909,165939.3530,165939.3530,165939.3530
1910,76251.6143,76251.6143,76251.6143
...,...,...,...
2016,109972.0670,109972.0670,109972.0670
2017,139323.9440,139323.9440,139323.9440
2018,83574.9239,83574.9239,83574.9239


#### Write out St. Vrain Annual Natural Flow Predictions to CSV or Excel (you can skip)

In [12]:
# write out un-scaled output dataframe of predicted St. Vrain values
user_output_filepath = "./stvrain_predicted_nf.csv" # <-- user input
ext = os.path.basename(user_output_filepath).split(".")[1]
if ext == "csv":
    SS_inverse.to_csv(user_output_filepath)
elif ext == "xlsx":
    SS_inverse.to_excel(user_output_filepath)
else:
    print("Warning output filepath must be a *.csv or *.xlsx path")

display(f"Wrote output dataframe to {user_output_filepath}")

'Wrote output dataframe to ./stvrain_predicted_nf.csv'

#### Write out St. Vrain Annual Natural Flow Predictions in RiverWare DMI format (you can skip)

In [13]:
# write out un-scaled output dataframe of predicted St. Vrain values using RW format
scenario_output_dirname = "./StressTest" # <-- user input
stvrain_filename = "TMD_East_Slope_Supply.St_Vrain_Annual_Flow" # <-- user input
start_date_str = "start_date: 2024-12-31 24:00" # <-- user input
units_str = "units: acre-ft" # <-- user input

if not os.path.isdir(scenario_output_dirname):
    os.mkdir(scenario_output_dirname)
for indx, col in enumerate(SS_inverse.columns.tolist()):
    col_to_folder_rename = f"trace{indx+1}"
    print(f"Column {col} in SS_inverse is mapped to the folder {col_to_folder_rename}")
    write_date = SS_inverse[col].values
    temp_dirpath = os.path.join(scenario_output_dirname, col_to_folder_rename)
    if not os.path.isdir(temp_dirpath):
        os.mkdir(temp_dirpath)
    df_filepath = os.path.join(temp_dirpath, stvrain_filename)
    with open(df_filepath, "w+") as f:
        f.write(f"{start_date_str}\n")
        f.write(f"{units_str}\n")
        for v in write_date:
            f.write(f"{round(v,0)}\n")
    print(f"Column {col} written too {df_filepath}")
    print("\n")
        
    
    
    

Column NF_1 in SS_inverse is mapped to the folder trace1
Column NF_1 written too ./StressTest\trace1\TMD_East_Slope_Supply.St_Vrain_Annual_Flow


Column NF_2 in SS_inverse is mapped to the folder trace2
Column NF_2 written too ./StressTest\trace2\TMD_East_Slope_Supply.St_Vrain_Annual_Flow


Column NF_3 in SS_inverse is mapped to the folder trace3
Column NF_3 written too ./StressTest\trace3\TMD_East_Slope_Supply.St_Vrain_Annual_Flow


